텐서플로우 버전 2.x에서 작동할 수 있도록 코드 수정

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, MaxPool2D, AveragePooling2D, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# 데이터 로드
mnist = tf.keras.datasets.mnist

In [3]:
# 데이터 전처리
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # 데이터 정규화
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [4]:
# ResNet 블록 클래스 정의
class ResNetBlock(Layer):
    def __init__(self, num_filters, bottleneck_size, adjust_filters=False):
        super(ResNetBlock, self).__init__()
        self.adjust_filters = adjust_filters  # 입력과 출력 차원을 일치시키기 위한 선택적 차원 조정
        self.conv1 = Conv2D(bottleneck_size, (1, 1), activation='relu')  # 기존 코드의 1x1 conv (병목)
        self.conv2 = Conv2D(bottleneck_size, (3, 3), padding='same', activation='relu')  # 3x3 conv (병목 중간)
        self.conv3 = Conv2D(num_filters, (1, 1), activation='relu')  # 마지막 1x1 conv
        if self.adjust_filters:
            self.adjust_dims = Conv2D(num_filters, (1, 1), activation='relu')  # 출력 차원을 조정

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        if self.adjust_filters:
            inputs = self.adjust_dims(inputs)  # 입력 차원을 출력과 일치시킴
        return tf.nn.relu(x + inputs)  # 잔차 연결 (skip connection)

In [5]:
# 전체 모델 구성
class ResNetModel(Model):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.conv1 = Conv2D(64, (7, 7), strides=(2, 2), activation='relu', padding='same')  # 기존 코드의 초기 conv
        self.pool1 = MaxPool2D((3, 3), strides=(2, 2), padding='same')  # 초기 pooling
        self.block1 = ResNetBlock(128, 32, adjust_filters=True)
        self.block2 = ResNetBlock(256, 64, adjust_filters=True)
        self.block3 = ResNetBlock(512, 128, adjust_filters=True)
        self.block4 = ResNetBlock(1024, 256, adjust_filters=True)
        self.avg_pool = AveragePooling2D(pool_size=(4, 4))  # 평균 풀링 (기존 코드의 avg_pool과 일치)
        self.flat = Flatten()  # Flatten
        self.fc = Dense(10, activation='softmax')  # 최종 fully connected layer와 softmax (분류)

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.avg_pool(x)
        x = self.flat(x)
        return self.fc(x)

In [6]:
# 모델 인스턴스 생성 및 컴파일
model = ResNetModel()
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# 훈련 및 평가
# 기존 코드 epochs값 너무 커서 훈련에 많은 시간 소요: 100 -> 10 으로 변경
model.fit(x_train, y_train, epochs=10, batch_size=100) 
model.evaluate(x_test, y_test)

Epoch 1/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 253s 401ms/step - accuracy: 0.5434 - loss: 1.2805
Epoch 2/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 249s 415ms/step - accuracy: 0.9336 - loss: 0.2172
Epoch 3/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 253s 421ms/step - accuracy: 0.9587 - loss: 0.1354
Epoch 4/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 258s 430ms/step - accuracy: 0.9682 - loss: 0.1037
Epoch 5/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 245s 408ms/step - accuracy: 0.9728 - loss: 0.0850
Epoch 6/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 234s 390ms/step - accuracy: 0.9760 - loss: 0.0763
Epoch 7/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 233s 388ms/step - accuracy: 0.9791 - loss: 0.0666
Epoch 8/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 233s 389ms/step - accuracy: 0.9843 - loss: 0.0526
Epoch 9/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 236s 393ms/step - accuracy: 0.9841 - loss: 0.0491
Epoch 10/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 231s 385ms/step - accuracy: 0.9861 - loss: 0.0470
313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 53ms/step - accuracy: 0.9794 - loss: 0.0677


[0.0546768419444561, 0.9833999872207642]

In [18]:
import numpy as np
# 훈련 데이터, 검정 데이터의 오분류율
test_accuracy_accumulated = 0
train_accuracy_accumulated = 0
num_batches = 100

for _ in range(num_batches):
    # 테스트 데이터셋에서 배치 처리
    batch_index_test = np.random.choice(len(x_test), 100, replace=False)
    test_loss, test_accuracy = model.evaluate(x_test[batch_index_test], y_test[batch_index_test], verbose=0)
    test_accuracy_accumulated += test_accuracy / num_batches

    # 훈련 데이터셋에서 배치 처리
    batch_index_train = np.random.choice(len(x_train), 550, replace=False)
    train_loss, train_accuracy = model.evaluate(x_train[batch_index_train], y_train[batch_index_train], verbose=0)
    train_accuracy_accumulated += train_accuracy / num_batches

print("Test misclassification error:", 1 - test_accuracy_accumulated)
print("Training misclassification error:", 1 - train_accuracy_accumulated)

Test misclassification error: 0.016399989724159125
Training misclassification error: 0.01943634688854179
